<a href="https://colab.research.google.com/github/david900125/113NCHU_Big-Data-Analytics-for-Finance/blob/main/week10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

In [17]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
data = load_dataset('takala/financial_phrasebank', 'sentences_50agree')

In [19]:
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Example dataset
class SentenceDataset(Dataset):
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': self.labels[idx]
        }

encoded_inputs = tokenizer(data['train']['sentence'], padding=True, truncation=True, max_length=180, return_tensors="pt")
input_ids = encoded_inputs['input_ids'].to(device)
attention_mask = encoded_inputs['attention_mask'].to(device)
labels = torch.tensor(data['train']['label']).to(device)

dataset = SentenceDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=32)

# Step 4: Set up the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Step 5: Train the model
model.train()
for epoch in range(300):  # 3 epochs
    print(f"Epoch {epoch + 1}")
    for batch in dataloader:
      optimizer.zero_grad()
      outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
      loss = outputs.loss
      loss.backward()
      optimizer.step()
      print(f"Loss: {loss.item()}")


Using device: cuda


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1
Loss: 1.2772389650344849
Loss: 1.1692969799041748
Loss: 1.1845940351486206
Loss: 1.0157032012939453
Loss: 0.9892532229423523
Loss: 0.9287415146827698
Loss: 0.8915636539459229
Loss: 0.8428561687469482
Loss: 0.7871503233909607
Loss: 0.8443655967712402
Loss: 0.707846462726593
Loss: 0.7323739528656006
Loss: 1.130246877670288
Loss: 1.162138819694519
Loss: 1.0921449661254883
Loss: 1.1341168880462646
Loss: 1.1390148401260376
Loss: 0.8385685682296753
Loss: 0.8105887770652771
Loss: 0.8915607929229736
Loss: 0.8978675603866577
Loss: 1.316393256187439
Loss: 0.9611727595329285
Loss: 0.7072986364364624
Loss: 0.6638615131378174
Loss: 0.6633341312408447
Loss: 0.6188138127326965
Loss: 0.6913071274757385
Loss: 0.6271063089370728
Loss: 0.6151465177536011
Loss: 0.5847684144973755
Loss: 1.2242025136947632
Loss: 1.2831891775131226
Loss: 1.2893455028533936
Loss: 1.2957316637039185
Loss: 1.109534502029419
Loss: 1.3740007877349854
Loss: 1.2352033853530884
Loss: 1.2388297319412231
Loss: 1.24065732955932

In [20]:
model.eval()
test_texts = [
  "The company's profit has increased significantly this quarter.",
  "The increase in costs negatively affected the revenue.",
  "The company's performance remained stable."
]
test_inputs = tokenizer(test_texts, padding=True, truncation=True, max_length=180, return_tensors="pt")
test_inputs = {key: val.to(device) for key, val in test_inputs.items()}

# Get predictions
with torch.no_grad():
    outputs = model(**test_inputs)
    predictions = torch.argmax(outputs.logits, dim=1)
    label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}
    for prediction in predictions:
        print(label_map[prediction.item()])


Neutral
Neutral
Neutral
